In [ ]:
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.models import efficientnet_b0
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
dataset_dir = "/content/drive/MyDrive/Datasets"
batch_size = 32
num_epochs = 10
learning_rate = 1e-3
num_classes = 2
input_size = 224

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2,
                           contrast=0.2,
                           saturation=0.2,
                           hue=0.02),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
dataset_full = datasets.ImageFolder(root=dataset_dir, transform=None)
labels = dataset_full.targets
indices = np.arange(len(labels))
train_indices, val_indices = train_test_split(
    indices,
    test_size=0.2,
    random_state=25,
    stratify=labels
)

In [ ]:
train_dataset = Subset(dataset_full, train_indices)
val_dataset   = Subset(dataset_full, val_indices)

In [ ]:
class SubsetWithTransform(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, idx):
        x, y = self.subset[idx]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

In [ ]:
train_dataset = SubsetWithTransform(train_dataset, transform=train_transforms)
val_dataset   = SubsetWithTransform(val_dataset,   transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False)

In [ ]:
model = efficientnet_b0(weights="IMAGENET1K_V1")

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 126MB/s] 


In [ ]:
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, num_classes)

In [ ]:
model = model.to(device)

In [ ]:
def train_step(model, criterion, optimizer, dataloader, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * images.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total_samples += images.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects.double() / total_samples

    return epoch_loss, epoch_acc.item()


def valid_step(model, criterion, dataloader, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    total_samples = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total_samples += images.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc = running_corrects.double() / total_samples

    return epoch_loss, epoch_acc.item()

In [ ]:
def train(model, num_epochs, criterion, optimizer, scheduler, device):
    best_acc = 0.0
    best_model = None

    for epoch in tqdm(range(num_epochs), desc='Epoch'):

        train_loss, train_acc = train_step(model, criterion, optimizer, train_loader, device)
        val_loss, val_acc = valid_step(model, criterion, val_loader, device)

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val   Loss: {val_loss:.4f},   Val Acc:   {val_acc:.4f}")

        scheduler.step()

        if val_acc > best_acc:
            best_acc = val_acc
            best_model = model.state_dict()

    print(f"Best val Acc: {best_acc:.4f}")

    if best_model is not None:
        model.load_state_dict(best_model)

    torch.save(model.state_dict(), "efficientnet_crows_and_squirrels.pth")
    print("Модель сохранена в efficientnet_crows_and_squirrels.pth")

    return model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
model = train(model, num_epochs, criterion, optimizer, scheduler, device)

Epoch:  10%|█         | 1/10 [03:24<30:44, 204.96s/it]

Train Loss: 0.1107, Train Acc: 0.9574
Val   Loss: 0.0729,   Val Acc:   0.9942


Epoch:  20%|██        | 2/10 [03:39<12:24, 93.01s/it] 

Train Loss: 0.0471, Train Acc: 0.9868
Val   Loss: 0.0706,   Val Acc:   0.9766


Epoch:  30%|███       | 3/10 [03:54<06:40, 57.24s/it]

...

Train Loss: 0.0040, Train Acc: 1.0000
Val   Loss: 0.0306,   Val Acc:   0.9942


Epoch:  90%|█████████ | 9/10 [05:23<00:18, 18.19s/it]

Train Loss: 0.0039, Train Acc: 1.0000
Val   Loss: 0.0284,   Val Acc:   0.9942


Epoch: 100%|██████████| 10/10 [05:38<00:00, 33.80s/it]

Train Loss: 0.0046, Train Acc: 0.9985
Val   Loss: 0.0290,   Val Acc:   0.9942
Best val Acc: 1.0000
Модель сохранена в efficientnet_crows_and_squirrels.pth
